In [50]:
using LinearAlgebra
using Polynomials: Polynomial
using QuadGK
using Plots

In [64]:
# Função auxiliar para construir a matriz de Vandermonde

# -- Parâmetros --
# x: conjunto dos pontos do eixo x
# grau: grau que desejamos para ser uma das dimensões da matriz

function vandermonde(x, grau)
    n, = size(x) # quantidade de pontos
    V = zeros(n, grau + 1) # criamos uma matriz n X (grau + 1)
    
    # preenchemos a matriz criada
    for i = 1:n
        for j = 1:(grau + 1)
            V[i, j] = x[i] ^ (j - 1)
        end
    end
    
    return V
end

vandermonde (generic function with 2 methods)

In [15]:
function calcula_inversa(A)  
    n = length(A)
    if length(A) !== 1
        n, = size(A) # dimensão da matriz A
    end
    inv = zeros(n, n) # criamos uma matriz n X n para ser a inversa
    L, U = decomposicao_lu(A) # decompomos a matriz A na multiplicação de duas triangulares (uma inferior e outra superior)
    
    for i = 1:n
        # criamos uma vetor com a posição y[i] sendo 1 para que a junção de todos os y's em uma matriz corresponda à matriz identidade
        id = zeros(n)
        id[i] = 1
        
        # fazemos com que cada multiplicação seja igual a uma linha da identidade, pois sabemos que A * inv = I
        y = resolve_triangular_inferior(L, id) # resolvemos o sistema triangular inferior e obtemos y tal que Ly = b
        x = resolve_triangular_superior(U, y) # resolvemos o sistema triangular superior e obtemos x tal que Ux = y -> L(Ux) = b
        
        # formatamos o resultado na matriz inversa
        for j = 1:n
            inv[j, i] = x[j]
        end
    end
    
    return inv

end

calcula_inversa (generic function with 1 method)

In [25]:
function resolve_triangular_superior(U, b)   
    n = length(U)
    if length(U) !== 1
        n, = size(U) # dimensão da matriz U
    end
    x = zeros(n, 1) # criamos uma matriz coluna n x 1
    
    # resolução do sistema por substituição reversa
    for i = reverse(1:n)
        x[i] = b[i]
        for j = reverse(i + 1:n)
            x[i] -= U[i, j] * x[j]
        end
        x[i] /= U[i, i]
    end
    
    return x 
end

resolve_triangular_superior (generic function with 1 method)

In [26]:
# Função para resolver um sistema triangular inferior n x n por substituição direta

# -- Parâmetros --
# L: matriz triangular onde os elementos da diagonal são diferentes de zero
# b: matriz coluna de resultados

# -- Retorno --
# x: x tal que Lx = b

function resolve_triangular_inferior(L, b)
    n = length(L)
    if length(L) !== 1
        n, = size(L) # dimensão da matriz L
    end
    x = zeros(n, 1) # criamos uma matriz coluna n x 1
    
    # resolução do sistema por substituição direta
    for i = 1:n
        x[i] = b[i]
        for j = 1:i - 1
            x[i] -= L[i, j] * x[j]
        end
        x[i] /= L[i, i]
    end
    
    return x
end

resolve_triangular_inferior (generic function with 1 method)

In [16]:
function minimos_quadrados(A, b)
    At = transpose(A)
    x = calcula_inversa((At * A)) * At * b # calculamos a inversa usando decomposição LU para poupar custos
    # x = resolve_sistema_denso(At * A, At * b)
    return x
end

minimos_quadrados (generic function with 1 method)

In [18]:
function decomposicao_lu(A)
    #display(A)
    #println(length(A))
    #println(size(A))
    n = length(A)
    if length(A) !== 1
        n, = size(A) # dimensão da matriz A
    end
    #println(n)
    L = Matrix{Float64}(I, n, n) # L inicialmente eh uma matriz identidade n x n
    U = copy(A) # inicialmente criamos a matriz U como uma cópia da matriz densa A
    
    # percorremos a matriz U e L para preenche-las
    for i = 1:n 
        for j = i+1:n
            k = U[j, i] / U[i, i]
            L[j, i] = k
            U[j, :] -= k * U[i, :]
        end
    end
    
    return L, U  
end

decomposicao_lu (generic function with 1 method)

### Regressão com coeficientes lineares

_________________________________________________________________________________________________________________________________________________________________________________________

In [20]:
# Função auxiliar para construir a matriz de Vandermonde

# -- Parâmetros --
# x: conjunto dos pontos do eixo x
# grau: grau que desejamos para ser uma das dimensões da matriz

function vandermonde_para_linear(x, k_funcoes)
    n, = size(x) # quantidade de pontos
    k, = size(k_funcoes)
    V = zeros(n, k) # criamos uma matriz n+1 X k
    
    # preenchemos a matriz criada
    for i = 1:n
        for j = 1:(k)
            V[i, j] = k_funcoes[j](x[i]) 
        end
    end
    
    return V
end

vandermonde_para_linear (generic function with 1 method)

In [82]:
# Função para calcular a regressão polinomial com coeficientes lineares

# -- Parâmetros --
# x: conjunto dos pontos do eixo x
# y: conjunto dos pontos do eixo y
# funcoes: array de funções

# -- Retorno --
# Lista de coeficientes tal que F(x) = c_1*f_1(x)+...+c_k*f_k(x)

function regressão_coef_linear(x,y,funcoes)
    
    #Forma a matriz de vandermonde, aplicando uma função em cada coluna, tendo um x_n em cada linha.
    V=vandermonde_para_linear(x, funcoes)
 
    #Realiza a operação de mínimos quadrados e retorna um array de coeficientes.
    c = minimos_quadrados(V,y)
    return c
end

regressão_coef_linear (generic function with 1 method)

In [90]:
# -- Exemplo 1 --

x = [1.0, 3.0, 5]
y = [3, 5, 7.0]
f1(x)=1
f2(x)=x
f3(x)=x^2
f_array = [f1,f2,f3]
z = regressão_coef_linear(x,y,f_array)
fun(x)=z[1]*f1(x)+z[2]*f2(x)+z[3]*f3(x)

println("A lista de coeficientes encontrada foi: ", z)

# -- Checagem de resultados --  
fun(x) = z[1] * f1(x) + z[2] * f2(x) + z[3] * f3(x)  
n, = size(x) 
correto = true 

for i = 1:n     
    abs(fun(x[i]) - y[i]) < 0.0001 ? continue : correto = false 
end  
correto ? println("Os coeficientes encontrados estão corretos") : println("Os coeficientes encontrados estão incorretos")

A lista de coeficientes encontrada foi: [2.000000000000049, 0.9999999999999893, 2.886579864025407e-15]
Os coeficientes encontrados estão corretos


In [96]:
# -- Exemplo 2 --

x = [-3, 2, 5, 7, 10]
y = [-12, 9.2, 7.05, 8, 15.75]
f1(x)=1
f2(x)=x
f3(x)=x^2
f4(x)=x^3
f5(x)=x^4
f_array = [f1,f2,f3,f4,f5]
z = regressão_coef_linear(x,y,f_array)

println("A lista de coeficientes encontrada foi: ", z)

# -- Checagem de resultados --  
fun(x) = z[1] * f1(x) + z[2] * f2(x) + z[3] * f3(x)  + z[4]*f4(x) + z[5]*f5(x)
n, = size(x) 
correto = true 

for i = 1:n     
    abs(fun(x[i]) - y[i]) < 0.0001 ? continue : correto = false 
end  
correto ? println("Os coeficientes encontrados estão corretos") : println("Os coeficientes encontrados estão incorretos")

A lista de coeficientes encontrada foi: [8.026923076919912, 2.159198717950458, -1.0449711538458835, 0.13899358974345277, -0.004836538461529415]
Os coeficientes encontrados estão corretos


In [99]:
# -- Exemplo 3 --

x = [-20, -15, 0, 23, 33, 40, 80]
y = [-12, -2, 30, 50, 85.75, 132.2, 176]
f1(x)=1
f2(x)=x
f3(x)=x^2
f4(x)=x^3
f5(x)=x^4
f6(x)=x^5
f7(x)=x^6
f_array = [f1,f2,f3,f4,f5,f6,f7]
z = regressão_coef_linear(x,y,f_array)

println("A lista de coeficientes encontrada foi: ", z)

# -- Checagem de resultados --  
fun(x) = z[1] * f1(x) + z[2] * f2(x) + z[3] * f3(x) + z[4] * f4(x) + z[5] * f5(x) + z[6] * f6(x) + z[7] * f7(x)
n, = size(x) 
correto = true 

for i = 1:n     
    abs(fun(x[i]) - y[i]) < 0.0001 ? continue : correto = false 
end  
correto ? println("Os coeficientes encontrados estão corretos") : println("Os coeficientes encontrados estão incorretos")

A lista de coeficientes encontrada foi: [29.999999990622957, 1.2854079880801406, -0.06764433646331713, 0.0007601279803502744, 8.447330753867004e-5, -1.0386762150975067e-6, 1.1601689610349502e-10]
Os coeficientes encontrados estão corretos
